In [5]:
import os
import sys

sys.path.append("../src")
import localmodule


# Define constants.
models_dir = localmodule.get_models_dir()

In [6]:
os.listdir(models_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/vl1019/BirdVox-70k_models'